In [1]:
#Preprocessing GSE138266

# Librerie necessarie
library(tools)  # Per la funzione file_path_sans_ext
library(fs)     # Per le funzioni di gestione dei file

# Sezione per dividere il RAW output del dataset in cartelle con genes.tsv, barcodes.tsv, matrix.mtx da poter poi aprire con Read10X


# Definizione delle directory di input e output
dataDir <- "dataset/rawdataset/"
GEO <- "."
datasetDir <- file.path(dataDir, GEO, "GSE138266_RAW")
outputDir <- file.path(dataDir, GEO, "organized_data")

In [2]:
# Function to extract unique prefixes from file names
extract_prefix <- function(filename) {
  sub("(_barcodes.tsv.gz|_genes.tsv.gz|_matrix.mtx.gz)", "", filename)
}

In [3]:
files = list.files(datasetDir)
# Creazione della directory di output se non esiste
dir_create(outputDir)

# Funzione per copiare e rinominare i file nelle cartelle appropriate
organize_files <- function(datasetDir, files, outputDir) {
  prefixes <- unique(sapply(files, extract_prefix))
  
  for (single_prefix in prefixes) {
    
    barcode_file <- file.path(datasetDir, paste0(single_prefix, "_barcodes.tsv.gz"))
    gene_file <- file.path(datasetDir, paste0(single_prefix, "_genes.tsv.gz"))
    matrix_file <- file.path(datasetDir, paste0(single_prefix, "_matrix.mtx.gz"))
    print(barcode_file)
    
    # Crea la cartella se non esiste già
    folder_path <- file.path(outputDir, single_prefix)
    dir_create(folder_path, recurse = TRUE)
    
    # Rinomina e copia il file nella cartella corretta
    #new_filename <- paste0(prefix, ".", ext)
    file_copy(barcode_file, file.path(folder_path, "barcodes.tsv"), overwrite = TRUE)
    file_copy(gene_file, file.path(folder_path, "genes.tsv"), overwrite = TRUE)
    file_copy(matrix_file, file.path(folder_path, "matrix.mtx"), overwrite = TRUE)
    
    
  }
}

In [4]:
# Esegui la funzione per organizzare i file
organize_files(datasetDir,files, outputDir)

# Messaggio di completamento
cat("Organizzazione dei file completata.\n")

[1] "dataset/rawdataset//./GSE138266_RAW/GSM4104122_MS19270_CSF_GRCh38_barcodes.tsv.gz"
[1] "dataset/rawdataset//./GSE138266_RAW/GSM4104123_MS58637_CSF_GRCh38_barcodes.tsv.gz"
[1] "dataset/rawdataset//./GSE138266_RAW/GSM4104124_MS71658_CSF_GRCh38_barcodes.tsv.gz"
[1] "dataset/rawdataset//./GSE138266_RAW/GSM4104125_MS49131_CSF_GRCh38_barcodes.tsv.gz"
[1] "dataset/rawdataset//./GSE138266_RAW/GSM4104126_MS60249_CSF_GRCh38_barcodes.tsv.gz"
[1] "dataset/rawdataset//./GSE138266_RAW/GSM4104127_MS74594_CSF_GRCh38_barcodes.tsv.gz"
[1] "dataset/rawdataset//./GSE138266_RAW/GSM4104128_PST83775_CSF_GRCh38_barcodes.tsv.gz"
[1] "dataset/rawdataset//./GSE138266_RAW/GSM4104129_PTC32190_CSF_GRCh38_barcodes.tsv.gz"
[1] "dataset/rawdataset//./GSE138266_RAW/GSM4104130_PST95809_CSF_GRCh38_barcodes.tsv.gz"
[1] "dataset/rawdataset//./GSE138266_RAW/GSM4104131_PTC41540_CSF_GRCh38_barcodes.tsv.gz"
[1] "dataset/rawdataset//./GSE138266_RAW/GSM4104132_PST45044_CSF_GRCh38_barcodes.tsv.gz"
[1] "dataset/rawdataset//./

In [5]:
options(Seurat.object.assay.version = "v3")
options(Seurat.object.assay.calcn = TRUE)
library(Seurat)
library(Matrix)
library(dplyr)  # Ensure this is loaded for %>%
library(Azimuth)

read_10x_data <- function(folder_path) {
  # Chiamata a Read10X con i file corretti
  data <- Read10X(data.dir = folder_path)
  
  # Crea un oggetto Seurat con i dati letti
  seurat_obj <- CreateSeuratObject(counts = data, 
                                   assay = "RNA")
  
  return(seurat_obj)
}

subfolders <- list.dirs(outputDir, full.names = TRUE, recursive = FALSE)
print(length(subfolders))
seurat_list <- vector("list", length = length(subfolders))

#Inizializzo i tag che userò nel merge
prefixes <- unique(sapply(files, extract_prefix))
dataset_tags <- sapply(prefixes, function(prefix){
  split_prefix <- strsplit(prefix, "_")[[1]]
  patient_code <- split_prefix[[2]]
  sample_type <- split_prefix[[3]]
  paste(patient_code, sample_type, sep="_")
})
rm(prefixes)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.4.1 but the current version is
4.4.2; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.5 but the current
version is 1.7.2; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat



Attaching shinyBS



[1] 22


In [6]:
# Define the preprocessing function
preprocess_seurat <- function(seurat_obj, ns = 5) {

  seurat_obj <- subset(seurat_obj, subset = nFeature_RNA < 3000 & nCount_RNA <20000 & percent.mt < 6)
  
  return(seurat_obj)
}


In [7]:
# Define a function to process each dataset
process_dataset <- function(subfolder, dataset_tag, seurat_list, i) {
  cat("Currently processing dataset:", i, "\n")  # Use cat for better printing
  
  # Read the data from the current subfolder
  seurat_obj <- read_10x_data(subfolder)
  

  # Extract metadata from dataset_tag
  dataset_tag_split <- strsplit(dataset_tag, "_")[[1]]
  type <- dataset_tag_split[2]
  patient <- dataset_tag_split[1]
  
  # Determine patient class
  patient_class <- ifelse(grepl("^MS", patient), "MS",
                          ifelse(grepl("^PTC|^PST", patient), "CTRL", NA))
  
  # Add metadata to the Seurat object
  seurat_obj <- AddMetaData(seurat_obj, metadata = type, col.name = "Tissue.Type")
  seurat_obj <- AddMetaData(seurat_obj, metadata = patient, col.name = "Patient")
  seurat_obj <- AddMetaData(seurat_obj, metadata = patient_class, col.name = "Patient-Class")
  seurat_obj$orig.ident <- paste0(patient, patient_class, type)
  seurat_obj$dataset <- "gse138266"
  seurat_obj$nFeature_RNA <- colSums(seurat_obj@assays$RNA@counts > 0)
     seurat_obj$nCount_RNA <- colSums(seurat_obj@assays$RNA@counts)

    
  # Apply specific filtering for dataset 7
  if (i == 7) {
    seurat_obj <- subset(seurat_obj, subset = nCount_RNA > 2000 & nCount_RNA < 45000 & 
                         nFeature_RNA > 40 & nFeature_RNA < 5000)
  }
  
  # Calculate mitochondrial percentage
  seurat_obj[["percent.mt"]] <- PercentageFeatureSet(seurat_obj, pattern = "^MT-")


  # Print metadata for debugging
  print(head(seurat_obj@meta.data))
  
    
  # Preprocess the Seurat object
  seurat_obj <- preprocess_seurat(seurat_obj)
  
  # Run Azimuth for cell type annotation
  seurat_obj <- RunAzimuth(seurat_obj, ref = 'pbmcref')
  
  # Add the processed Seurat object to the list
  seurat_list[[i]] <- seurat_obj
  
  return(seurat_list)
}

# Main loop to process all subfolders
seurat_list <- list()  # Initialize list to store Seurat objects
for (i in seq_along(subfolders)) {
  tryCatch({
    seurat_list <- process_dataset(subfolders[i], dataset_tags[i], seurat_list, i)
  }, error = function(e) {
    cat("Error processing dataset", i, ":", e$message, "\n")  # Handle errors gracefully
  })
}

Currently processing dataset: 1 


Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"


                     orig.ident nCount_RNA nFeature_RNA Tissue.Type Patient
AAACCTGAGTGGGCTA-1 MS19270MSCSF       2566          651         CSF MS19270
AAACCTGAGTGTTAGA-1 MS19270MSCSF       2284          844         CSF MS19270
AAACCTGGTCGCGTGT-1 MS19270MSCSF      17837         3022         CSF MS19270
AAACCTGGTCTCCACT-1 MS19270MSCSF       3721         1058         CSF MS19270
AAACCTGGTTTACTCT-1 MS19270MSCSF       3422         1130         CSF MS19270
AAACCTGTCTCCTATA-1 MS19270MSCSF       4175         1159         CSF MS19270
                   Patient-Class   dataset percent.mt
AAACCTGAGTGGGCTA-1            MS gse138266   2.455183
AAACCTGAGTGTTAGA-1            MS gse138266   2.977233
AAACCTGGTCGCGTGT-1            MS gse138266   2.657398
AAACCTGGTCTCCACT-1            MS gse138266   2.499328
AAACCTGGTTTACTCT-1            MS gse138266   3.535944
AAACCTGTCTCCTATA-1            MS gse138266   1.940120


Warning message:
"Overwriting miscellanous data for model"
Warning message:
"Adding a dimensional reduction (refUMAP) without the associated assay being present"
Warning message:
"Adding a dimensional reduction (refUMAP) without the associated assay being present"
detected inputs from HUMAN with id type Gene.name

reference rownames detected HUMAN with id type Gene.name

Normalizing query using reference SCT model

Warning message:
"735 features of the features specified were not present in both the reference query assays. 
Continuing with remaining 4265 features."
Projecting cell embeddings

Counts matrix provided is not sparse; vreating v5 assay in Seurat object

Finding query neighbors

Finding neighborhoods

Finding anchors

	Found 4317 anchors

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Predicting cell labels

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Predicting cell labels

Warning mes

Currently processing dataset: 2 


...

Warning message:
"Overwriting miscellanous data for model"
Warning message:
"Adding a dimensional reduction (refUMAP) without the associated assay being present"
Warning message:
"Adding a dimensional reduction (refUMAP) without the associated assay being present"
detected inputs from HUMAN with id type Gene.name

reference rownames detected HUMAN with id type Gene.name

Normalizing query using reference SCT model

Warning message:
"735 features of the features specified were not present in both the reference query assays. 
Continuing with remaining 4265 features."
Projecting cell embeddings

Counts matrix provided is not sparse; vreating v5 assay in Seurat object

Finding query neighbors

Finding neighborhoods

Finding anchors

	Found 4902 anchors

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Predicting cell labels

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Predicting cell labels

Warning mes

Currently processing dataset: 22 


Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"


                          orig.ident Tissue.Type  Patient Patient-Class
AAACCTGAGCGTGTCC-1 PTC85037CTRLPBMCs       PBMCs PTC85037          CTRL
AAACCTGAGGTCATCT-1 PTC85037CTRLPBMCs       PBMCs PTC85037          CTRL
AAACCTGAGTGGACGT-1 PTC85037CTRLPBMCs       PBMCs PTC85037          CTRL
AAACCTGTCATCGGAT-1 PTC85037CTRLPBMCs       PBMCs PTC85037          CTRL
AAACGGGAGAGGTAGA-1 PTC85037CTRLPBMCs       PBMCs PTC85037          CTRL
AAACGGGAGGTTCCTA-1 PTC85037CTRLPBMCs       PBMCs PTC85037          CTRL
                     dataset nFeature_RNA nCount_RNA percent.mt
AAACCTGAGCGTGTCC-1 gse138266           40        964  0.1037344
AAACCTGAGGTCATCT-1 gse138266          832       3269  2.6613643
AAACCTGAGTGGACGT-1 gse138266          814       3279  4.6050625
AAACCTGTCATCGGAT-1 gse138266          833       2806  5.8802566
AAACGGGAGAGGTAGA-1 gse138266          785       3027  4.6580773
AAACGGGAGGTTCCTA-1 gse138266          816       2060  4.2233010


Warning message:
"Overwriting miscellanous data for model"
Warning message:
"Adding a dimensional reduction (refUMAP) without the associated assay being present"
Warning message:
"Adding a dimensional reduction (refUMAP) without the associated assay being present"
detected inputs from HUMAN with id type Gene.name

reference rownames detected HUMAN with id type Gene.name

Normalizing query using reference SCT model

Warning message:
"735 features of the features specified were not present in both the reference query assays. 
Continuing with remaining 4265 features."
Projecting cell embeddings

Counts matrix provided is not sparse; vreating v5 assay in Seurat object

Finding query neighbors

Finding neighborhoods

Finding anchors

	Found 2946 anchors

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Predicting cell labels

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Predicting cell labels

Warning mes

In [9]:
merged_data <- merge(seurat_list[[1]], y = seurat_list[-1])
print(merged_data)

Warning message:
"Some cell names are duplicated across objects provided. Renaming to enforce unique cell names."


An object of class Seurat 
33789 features across 74735 samples within 4 assays 
Active assay: RNA (33694 features, 0 variable features)
 2 layers present: counts, data
 3 other assays present: prediction.score.celltype.l1, prediction.score.celltype.l2, prediction.score.celltype.l3


In [10]:
only_pbmc <- subset(merged_data, subset=Tissue.Type=="PBMCs")

In [ ]:
only_pbmc

In [ ]:
# CONTROLLO A CAMPIONE DI ALCUNI DI QUESTI GENI
genes_test<- FetchData(only_pbmc, vars = c("OR4F29", "OR2M7", "REN", "C1orf195") )
# sono tutti zero. li elimino

# Prendo le varianze
expr_matrix <- GetAssayData(only_pbmc, slot = "counts")
gene_variances <- apply(expr_matrix, 1, var)
gene_variance_df <- data.frame(Gene = rownames(expr_matrix), Variance = gene_variances)
nonzero_variance_genes <- names(gene_variances)[gene_variances > 0] # Questi sono i geni la cui varianza è maggiore di zero

In [ ]:
only_pbmc_pre_no_zero_var <- subset(only_pbmc, features = nonzero_variance_genes)

In [11]:
only_pbmc <- NormalizeData(only_pbmc)
all.genes <- rownames(only_pbmc)
only_pbmc <- ScaleData(only_pbmc, features = all.genes)

Centering and scaling data matrix



In [ ]:
only_pbmc_pre_no_zero_var <- RunPCA(only_pbmc_pre_no_zero_var, features = all.genes)

In [ ]:
only_pbmc_pre_no_zero_var <- RunUMAP(only_pbmc_pre_no_zero_var, dims = 1:10)

In [ ]:
DimPlot(only_pbmc_pre_no_zero_var, reduction = "umap", group.by="orig.ident", label=TRUE)

In [12]:
library(Seurat)
library(SeuratDisk)

# Supponendo che `gse144744` sia un oggetto Seurat
SaveH5Seurat(only_pbmc, filename = "gse138266_pbmc.h5Seurat")
Convert("gse138266_pbmc.h5Seurat", dest = "h5ad")


Attaching package: 'SeuratDisk'


The following object is masked from 'package:Azimuth':

    Connect


Creating h5Seurat file for version 3.1.5.9900

Adding counts for RNA

Adding data for RNA

Adding scale.data for RNA

No variable features found for RNA

No feature-level metadata found for RNA

Adding counts for prediction.score.celltype.l1

Adding data for prediction.score.celltype.l1

No variable features found for prediction.score.celltype.l1

No feature-level metadata found for prediction.score.celltype.l1

Adding counts for prediction.score.celltype.l2

Adding data for prediction.score.celltype.l2

No variable features found for prediction.score.celltype.l2

No feature-level metadata found for prediction.score.celltype.l2

Adding counts for prediction.score.celltype.l3

Adding data for prediction.score.celltype.l3

No variable features found for prediction.score.celltype.l3

No feature-level metadata found for prediction.score.celltype.l3

Validating h5Seurat file

Adding scal